# Рекомендация тарифов

## Откройте и изучите файл

### Импортируем необходимые библиотеки и загружаем датасет

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier

In [4]:
try:
    data = pd.read_csv('/datasets/users_behavior.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')

### Выведим первые 20 строк датасета 

In [3]:
data.head(20)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


### Выведим общую информацию про датасет

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


###  Посмотрим на статистическую информацию по столбцам

In [5]:
data.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


### Изменим типы данных

In [6]:
data['calls'] = data['calls'].astype('int')
data['messages'] = data['messages'].astype('int')

**Вывод** <br>
Видем, что в данных нету пропусков, данных довольно много, нету аномальных значений. Так же мы изменили типы данных в некоторых столбцах на более подходящие, а значит, можно переходить к исследованию

## Разбейте данные на выборки

### Разделим данные на обучающую, валидационную и тестовую выборки в соотношении 3:1:1

In [5]:
data_train, data_check = train_test_split(data, test_size=0.4, random_state=12345)
data_valid, data_test = train_test_split(data_check, test_size=0.5, random_state=12345)

### Разделим данные по признакам и целевому признаку

In [6]:
features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']

features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']

features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']

In [7]:
target_train

3027    0
434     0
1226    0
1054    0
1842    0
       ..
2817    1
546     1
382     1
2177    1
482     1
Name: is_ultra, Length: 1928, dtype: int64

**Вывод** <br>
Разбили данные на обучающую, валидационную и тестовую выборки в соотношении 3:1:1. Раделили данные по признакам и целевому признаку

## Исследуйте модели

### Модель "Решающее дерево"

Исследуем модель "Решающее дерево". С помощью цикла будем менять гиперпараметр max_depth от 1 до 10, и посмотрим в каком случае будет лучший показатель accuracy

In [9]:
best_model_tree = None
best_result_tree = 0
best_depth_tree = 0
best_criterion = None

criterions = {'gini', 'entropy'}

for crit in criterions:
    for depth in range(1, 11):
        model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth, criterion=crit) 
        model_tree.fit(features_train, target_train) 
        predictions = model_tree.predict(features_valid) 
        result = accuracy_score(target_valid, predictions) 
        if result > best_result_tree:
            best_model_tree = model_tree
            best_result_tree = result
            best_depth_tree = depth
            best_criterion = crit
        
print(f'Accuracy наилучшей модели на валидационной выборке: {best_result_tree}, при max_depth = {best_depth_tree}, \
при criterion = {best_criterion}') 

Accuracy наилучшей модели на валидационной выборке: 0.7853810264385692, при max_depth = 3, при criterion = entropy


### Модель "Случайный лес"

Исследуем модель "Случайный лес". С помощью цикла будем менять гиперпараметр max_depth от 1 до 10, и гиперпараметр n_estimators от 1 до 15. Найдём лучшие значения этих гиперпараметров и выведим лучшее accuracy

In [10]:
best_model_forest = None
best_result_forest = 0
best_depth_forest = 0
best_n_estimators_forest = 0
best_criterion = None

criterions = {'gini', 'entropy'}

for crit in criterions:
    for depth in range(1, 11):
        for est in range(1, 51):
            model_forest = RandomForestClassifier(random_state=12345, max_depth = depth, n_estimators=est, criterion=crit) 
            model_forest.fit(features_train, target_train) 
            result = model_forest.score(features_valid, target_valid) 
            if result > best_result_forest:
                best_model_forest = model_forest
                best_result_forest = result
                best_depth_forest = depth
                best_n_estimators_forest = est
                best_criterion = crit

print(f'Accuracy наилучшей модели на валидационной выборке: {best_result_forest}, при max_depth = {best_depth_tree}, \
при n_estimators = {best_n_estimators_forest}, при criterion = {best_criterion}') 

Accuracy наилучшей модели на валидационной выборке: 0.8087091757387247, при max_depth = 3, при n_estimators = 40, при criterion = gini


### Модель "Логистическая регрессия"

Исследуем модель "Логическая регрессия". Здесь нету гиперпараметров, которые бы влияли на результат.

In [11]:
best_model_lr = None
best_result_lr = 0
best_solver = None
best_c = 0

solvers = {'liblinear', 'lbfgs'}

for c in range(1, 11):
    for sol in solvers:
        model_lr = LogisticRegression(random_state=12345, solver=sol, C=c)
        model_lr.fit(features_train, target_train)
        result = model_lr.score(features_valid, target_valid)
        if result > best_result_lr:
                    best_model_lr = model_lr
                    best_result_lr = result 
                    best_solver = sol
                    best_c = c
                

print(f'Accuracy модели на валидационной выборке: {best_result_lr}, при solver = {best_solver}, при c = {best_c}')

Accuracy модели на валидационной выборке: 0.7573872472783826, при solver = liblinear, при c = 5


**Вывод** <br>
- Лучше всего на валидационной выборке себя показала модель "Случайный лес". У неё получился наилучший показатель accuracy - 0.8. 
- На втором месте оказалась модель "Решающее дерево" с показателем accuracy - 0.76. 
- Ну и хуже всех себя проявила модель "Логистическая регрессия" с показателем accuracy - 0.75.

## Проверьте модель на тестовой выборке

In [12]:
test_result = best_model_forest.score(features_test, target_test)

print(f'Accurancy модели "Случайный лес" на тестовой выборке: {test_result}')

Accurancy модели "Случайный лес" на тестовой выборке: 0.7962674961119751


**Вывод** <br>
Точность лучшей модели "Случайный лес" на тестовой выборке составила 0.796, что является довольно хорошим результатом

## (бонус) Проверьте модели на адекватность

### Первый способ

Чтобы оценить вменяемость нашей модели, сравним её результат со случайными предсказаниями. 

In [13]:
random_answers = np.random.randint(0, 2, size=643)

accuracy_random_test = accuracy_score(target_test, random_answers)
accuracy_random_test

0.49611197511664074

**Вывод** <br>
Как мы видим, случайные предсказания имеют показатель схожести с тестовой выборкой около 0.5, значит наша модель, имеющая результат около 0.8, вполне точная, а значит адекватная

### Второй способ (Dummy Models)

In [29]:
strategies = ['most_frequent', 'stratified', 'uniform', 'constant']
  
test_scores = []

for s in strategies:
    if s =='constant':
        dclf = DummyClassifier(strategy = s, random_state = 12345, constant=1)
    else:
        dclf = DummyClassifier(strategy = s, random_state = 12345)
    dclf.fit(features_train, target_train)
    score = dclf.score(features_test, target_test)
    test_scores.append(score)

print(f'При strategy = {strategies[0]}, score = {test_scores[0]}')
print(f'При strategy = {strategies[1]}, score = {test_scores[1]}')
print(f'При strategy = {strategies[2]}, score = {test_scores[2]}')
print(f'При strategy = {strategies[3]}, score = {test_scores[3]}')

При strategy = most_frequent, score = 0.6842923794712286
При strategy = stratified, score = 0.536547433903577
При strategy = uniform, score = 0.4821150855365474
При strategy = constant, score = 0.3157076205287714


## Вывод

**В ходе исследования было сделано:** <br>
1. Ознакомились с данными, посмотрели на общую информацию, поменяли типы данных в некоторых столбцах
2. Разделили данные на обучающую, валидационную и тестовую выборки
3. Разделили данные на признаки и таргеты
4. Исследовали модель "Решающее дерево"
5. Исследовали модель "Случайный лес"
6. Исследовали модель "Логистическая регрессия"
7. Проверили лучшию модель "Случайный лес" на тестовой выборке
8. Проверили модель на адекватность